# BRICS cleaning data

#### This script contains

    01 Data Wrangling
    02 Quality and Consistency Checks
    03 Export of world_hapiness_cleaned1.csv

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import os
import sklearn
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
path = r'C:\Users\Lenovo\OneDrive\Dokumente\CareerFoundry\Data Immersion\6. Advanced Analytics and Dashboard Design'

In [3]:
# Import Data
df = pd.read_csv(os.path.join(path, 'BRICS', '02 Data', 'Original Data', 'Life_Expectancy_00_20.csv'))


## 01 Data wrangling

    Renaming columns
    

###  Renaming columns


In [4]:
# checking shape
df.shape

(2469, 17)

In [5]:
# checking column names
df.columns

Index(['Country', 'Year', 'Continent', 'Least Developed', 'Life Expectancy',
       'Population', 'CO2 emissions', 'Health expenditure',
       'Electric power consumption', 'Forest area', 'GDP per capita',
       'Individuals using the Internet', 'Military expenditure',
       'People practicing open defecation',
       'People using at least basic drinking water services',
       'Obesity among adults', 'Beer consumption per capita'],
      dtype='object')

In [6]:
# renaming columns
df.rename(columns={'Country' : 'country', 'Year' : 'year', 'People using at least basic drinking water services' : 'drinking_water', 'People practicing open defecation' : 'open_defecation', 'Continent' : 'continent', 'Least Developed' : 'least_developed', 'Life Expectancy' : 'life_expectancy', 'Population' : 'population', 'CO2 emissions' : 'co2_emissions', 'Health expenditure' : 'health_expenditure', 'Electric power consumption' : 'electric_power_consumption', 'Forest area' : 'forest_area', 'GDP per capita' : 'gdp_per_capita', 'Individuals using the Internet' : 'internet_users', 'Military expenditure' : 'military_exp', 'Obesity among adults' : 'obesity_among_adults', 'Beer consumption per capita' : 'beer_cons_per_capita' }, inplace = True)


In [7]:
# checking data types
df.dtypes


country                        object
year                            int64
continent                      object
least_developed                  bool
life_expectancy               float64
population                      int64
co2_emissions                 float64
health_expenditure            float64
electric_power_consumption    float64
forest_area                   float64
gdp_per_capita                float64
internet_users                float64
military_exp                  float64
open_defecation               float64
drinking_water                float64
obesity_among_adults          float64
beer_cons_per_capita          float64
dtype: object

#### No need for changing data types nor Transposing data

## 02 Quality & Consistency Checks

    quality checks
    consistency checks

### quality checks

#### mixed type variables

In [8]:
# Überprüfung auf gemischte Datentypen
for col in df.columns.tolist():
    weird = (df[[col]].apply(lambda x: type(x.iloc[0])) != df[[col]].applymap(type)).any(axis=1)
    if len(df[weird]) > 0:
        print(col)

year
least_developed
life_expectancy
population
co2_emissions
health_expenditure
electric_power_consumption
forest_area
gdp_per_capita
internet_users
military_exp
open_defecation
drinking_water
obesity_among_adults
beer_cons_per_capita


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_106748\4181605260.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df[[col]].apply(lambda x: type(x.iloc[0])) != df[[col]].applymap(type)).any(axis=1)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_106748\4181605260.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df[[col]].apply(lambda x: type(x.iloc[0])) != df[[col]].applymap(type)).any(axis=1)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_106748\4181605260.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df[[col]].apply(lambda x: type(x.iloc[0])) != df[[col]].applymap(type)).any(axis=1)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_106748\4181605260.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  weird = (df[[col]].apply(lambda x: type(x.iloc[0])) != df[[col]].applymap(type)).any(axis=1)
C:\Users\Lenovo\

#### missing values


In [9]:
# Check for columns with missing values
df.isnull().sum()


country                         0
year                            0
continent                       0
least_developed                 0
life_expectancy                 0
population                      0
co2_emissions                   0
health_expenditure              9
electric_power_consumption    565
forest_area                     0
gdp_per_capita                  0
internet_users                  0
military_exp                    0
open_defecation                 0
drinking_water                  0
obesity_among_adults          452
beer_cons_per_capita          117
dtype: int64

In [10]:
# checking missing values by year
df.groupby(['year']).country.nunique()

year
2000    119
2001    119
2002    119
2003    119
2004    119
2005    119
2006    119
2007    119
2008    119
2009    119
2010    119
2011    119
2012    119
2013    119
2014    119
2015    119
2016    113
2017    113
2018    113
2019    113
2020    113
Name: country, dtype: int64

In [11]:
#dataframe from 2016 to 2020
df_missing=df[df['year'].isin([2016,2017,2018,2019,2020])]

In [12]:
# list of countries that have no data
y=set(df['country'].unique())-set(df_missing['country'].unique())#
y

{'Congo Dem. Rep.', 'Congo Rep.', 'Egypt', 'Iran', 'Venezuela', 'Yemen'}

In [13]:
# delete coutries with missing values
df = df[~df['country'].isin(y)]


In [14]:
for country in set(df['country'].unique()):
    mean1 = df.loc[df["country"] == country, 'beer_cons_per_capita'].mean()
    mean2 = df.loc[df["country"] == country, 'electric_power_consumption'].mean()
    mean3 = df.loc[df["country"] == country, 'obesity_among_adults'].mean()
    mean4 = df.loc[df["country"] == country, 'health_expenditure'].mean()

    df.loc[df["country"] == country, 'beer_cons_per_capita'].fillna(mean1, inplace=True)
    df.loc[df["country"] == country, 'electric_power_consumption'].fillna(mean2, inplace=True)
    df.loc[df["country"] == country, 'obesity_among_adults'].fillna(mean3, inplace=True)
    df.loc[df["country"] == country, 'health_expenditure'].fillna(mean4, inplace=True)

In [15]:
df['country'].value_counts()

country
Albania     21
Libya       21
Peru        21
Paraguay    21
Panama      21
            ..
Finland     21
Ethiopia    21
Estonia     21
Eritrea     21
Zimbabwe    21
Name: count, Length: 113, dtype: int64

In [16]:
# Check for columns with missing values
df.isnull().sum()

# No missing values

country                         0
year                            0
continent                       0
least_developed                 0
life_expectancy                 0
population                      0
co2_emissions                   0
health_expenditure              9
electric_power_consumption    565
forest_area                     0
gdp_per_capita                  0
internet_users                  0
military_exp                    0
open_defecation                 0
drinking_water                  0
obesity_among_adults          452
beer_cons_per_capita          117
dtype: int64

#### duplicate values

In [17]:
# creating a subset with duplicates
df_dups = df[df.duplicated()]

In [18]:
# checking for values in the subset
df_dups

# No duplicates found

,country,year,continent,least_developed,life_expectancy,population,co2_emissions,health_expenditure,electric_power_consumption,forest_area,gdp_per_capita,internet_users,military_exp,open_defecation,drinking_water,obesity_among_adults,beer_cons_per_capita


### Consistency checks

#### value counts for categorical columns 

In [19]:
df['country'].value_counts(dropna=False)

country
Albania     21
Libya       21
Peru        21
Paraguay    21
Panama      21
            ..
Finland     21
Ethiopia    21
Estonia     21
Eritrea     21
Zimbabwe    21
Name: count, Length: 113, dtype: int64

In [20]:
df['continent'].nunique()

6

In [21]:
df['continent'].value_counts(dropna=False)

continent
Europe           798
Asia             588
Africa           525
North America    231
South America    189
Oceania           42
Name: count, dtype: int64

In [22]:
df['least_developed'].value_counts(dropna=False)

least_developed
False    2100
True      273
Name: count, dtype: int64

In [23]:
df.columns

Index(['country', 'year', 'continent', 'least_developed', 'life_expectancy',
       'population', 'co2_emissions', 'health_expenditure',
       'electric_power_consumption', 'forest_area', 'gdp_per_capita',
       'internet_users', 'military_exp', 'open_defecation', 'drinking_water',
       'obesity_among_adults', 'beer_cons_per_capita'],
      dtype='object')

## 03 EXPORT world_hapiness_cleaned1.csv

In [24]:
# save the  dataframe as a csv in prepeared data folder
df.to_csv(os.path.join(path, 'BRICS', '02 Data', 'Prepared Data', 'world_hapiness_cleaned1.csv'))
